In [ ]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
import glob
import os
pd.options.display.float_format = '{:,.2f}'.format

# READING FILES

In [ ]:
date_respuesta = '2021-04-29'
date_respuesta = pd.to_datetime(date_respuesta)

## FP DOWNLOAD 

In [ ]:
name_header=['Nombre','Blank','amount','CLABE','report_id']

In [ ]:
ae = glob.glob(r'C:\Users\Cristian.Aparicio\Documents\FAST PAYMENTS\2021\04. ABRIL\FPDOWNLOAD*.csv', recursive=False)
ae

In [ ]:
ae_concat = pd.concat([pd.read_csv(f,names=name_header,dtype={'CLABE':str,'report_id':str}
                            ,skiprows=[0]).assign(file_name=os.path.basename(f)) for f in ae])
ae_concat.loc[ae_concat['file_name'].str.contains('FPDOWNLOAD-1'), 'Time'] = '09:00'
ae_concat.loc[ae_concat['file_name'].str.contains('FPDOWNLOAD-2'), 'Time'] = '12:00'
ae_concat.loc[ae_concat['file_name'].str.contains('FPDOWNLOAD-3'), 'Time'] = '16:00'
ae_concat.loc[ae_concat['file_name'].str.contains('FPDOWNLOAD-4'), 'Time'] = '21:00'
ae_concat.head()

In [ ]:
ae_concat.shape

In [ ]:
ae_concat['query'] = "'"+ae_concat['report_id'].str[5:].replace(' ', '',regex=True)+"',"
print (ae_concat['query'].to_string(index=False))

## BIN

In [ ]:
bin1 = pd.read_csv(r'C:\Users\Cristian.Aparicio\Documents\DISPERSIONES\Catalogo_Bancos.csv'
                  ,dtype={'BIN':str,'Clave de la institución':str,'Nombre de la institución':str})


In [ ]:
bin1.head()

In [ ]:
bin1.shape

## RESPUESTAS FP

In [ ]:
rs = glob.glob(r'C:\Users\Cristian.Aparicio\Documents\FAST PAYMENTS\2021\04. ABRIL\Respuesta-FP*.xlsx', recursive=False)
rs

In [ ]:
rs_concat = pd.concat([pd.read_excel(f,sheet_name='Sheet1',converters={'dsp_reference_id':str,'bank_account_number':str
                                                                     ,'disbursement_tracking_key':str,'report_id':str})
                       .assign(file_name=os.path.basename(f)) for f in rs])
rs_concat['CLABE'] = rs_concat['bank_account_number']
rs_concat.loc[rs_concat['file_name'].str.contains('FP-1'), 'Time'] = '09:00'
rs_concat.loc[rs_concat['file_name'].str.contains('FP-2'), 'Time'] = '12:00'
rs_concat.loc[rs_concat['file_name'].str.contains('FP-3'), 'Time'] = '16:00'
rs_concat.loc[rs_concat['file_name'].str.contains('FP-4'), 'Time'] = '21:00'
rs_concat['BIN'] = rs_concat['bank_account_number'].str[:3].replace(' ', '',regex=True)
rs_concat.head()

In [ ]:
rs_concat.shape

In [ ]:
duplicates = rs_concat[rs_concat.duplicated(['report_id'] , keep=False)]
duplicates.head()

In [ ]:
duplicates.shape

In [ ]:
rs_concat_2 = rs_concat.merge(bin1[['BIN','Clave de la institución','Nombre de la institución']] ,on='BIN', how='left')
rs_concat_2.head()

In [ ]:
rs_concat.shape

## QUERY PAYMENTS

In [ ]:
disp1 = pd.read_csv(r'C:\Users\Cristian.Aparicio\Documents\FAST PAYMENTS\2021\04. ABRIL\FP 2021-04-29.csv')
disp1.head()

In [ ]:
disp1.shape

# ARCHIVO RESPUESTA

In [ ]:
rs_concat_2['Date_respuesta'] = date_respuesta
rs_concat_2['Metodo de dispersion'] = 'BANORTE'
rs_concat_2.head()

In [ ]:
rs_concat.shape

In [ ]:
rs_concat_2['Banco emisor'] = '40072'
rs_concat_2['Comision'] = ''
rs_concat_2['LINEA CEP'] = rs_concat_2['Date_respuesta'].astype(str).replace(' ', '',regex=True)+','+rs_concat_2['disbursement_tracking_key'].astype(str).replace(' ', '',regex=True)+','+rs_concat_2['Banco emisor'].astype(str).replace(' ', '',regex=True)+','+rs_concat_2['Clave de la institución'].astype(str).replace(' ', '',regex=True)+','+rs_concat_2['bank_account_number'].astype(str).replace(' ', '',regex=True)+','+rs_concat_2['amount'].astype(str).replace(' ', '',regex=True)
rs_concat_2.head()

In [ ]:
rs_concat_2.shape

# ARCHIVO DISPERSION

In [ ]:
disp1.head()

In [ ]:
disp1['report_date_local'] = pd.to_datetime(disp1['report_date_local'])

In [ ]:
split = disp1["swap"].str.split(",",n=7,expand=True)
split.head()

In [ ]:
disp1["swap"] = split[0] 
disp1["CLABE"] = split[3]

In [ ]:
disp1.head()

In [ ]:
disp1.loc[disp1['report_date_local'] == date_respuesta, 'tipo'] = 'Pago del día'
disp1.loc[disp1['report_date_local'] != date_respuesta, 'tipo'] = 'Reprogramación'
disp1.drop(['bank_name','CLABE','status'],axis=1,inplace=True)
disp1.head()

In [ ]:
disp_2 = disp1.merge(rs_concat_2[['report_id','CLABE','status','Nombre de la institución','LINEA CEP','Time']]
                     ,on='report_id', how='left')
disp_2.sort_values(by=['status'])
disp_2["Time"].fillna('NO TIME',inplace=True)
disp_2.loc[disp_2['Time'] == 'NO TIME', 'status'] = 'ERROR'
disp_2.head()

In [ ]:
disp_2.loc[disp_2['Nombre de la institución'] == 'BANORTE/IXE', 'LINEA CEP'] = 'COMPROBANTE MISMO BANCO BANORTE'
disp_2.loc[disp_2['status'] == 'ERROR', 'LINEA CEP'] = ''

In [ ]:
for col in disp_2.columns: 
    print(col) 

In [ ]:
disp_final = disp_2[['merchant_id','merchant_name','user_email','report_id','report_date_local','amount','tipo'
                    ,'Nombre de la institución','swap','CLABE','status','LINEA CEP','Time']]
disp_final.rename(columns ={'Nombre de la institución':'bank','amount':'total_due','LINEA CEP':'CEP'}, inplace=True)
disp_final.sort_values(by=['status','Time'], ascending=True, inplace=True)
disp_final.drop_duplicates(subset =['report_id','Time'], keep ='first', inplace=True)
disp_final.head()

In [ ]:
disp_final.shape

# ERROR

In [ ]:
error = disp_final[(disp_final['status'] != 'PAID')]
error.head()

In [ ]:
error.shape

In [ ]:
error.drop(['swap','CLABE','status','CEP'],axis=1,inplace=True)

In [ ]:
error_final = error
error_final.head()

In [ ]:
error.loc[error['bank'] == 'BANORTE/IXE', 'Check'] = 'SPEI DEVUELTO MISMO BANCO'
error.loc[error['bank'] != 'BANORTE/IXE', 'Check'] = 'SPEI DEVUELTO'

In [ ]:
error.shape

In [ ]:
error_final = error.merge(rs_concat_2[['report_id','error_description']],on='report_id',how='left')
error_final.head()

In [ ]:
error_final.shape

# ARCHIVOS FINALES

### ARCHIVO DISPERSION

In [ ]:
writer = pd.ExcelWriter(r'C:\Users\Cristian.Aparicio\Documents\FAST PAYMENTS\2021\04. ABRIL\ARCHIVO DISPERSIONES FP\Dispersión-FP 2021-04-29.xlsx', engine='xlsxwriter', options={'encoding':'utf-8'})
disp_final.to_excel(writer, sheet_name='FAST PAYMENTS',index=False)
error_final.to_excel(writer, sheet_name='ERROR',index=False)
writer.save()
writer.close()

### ARCHIVO RESPUESTA 

In [ ]:
rs_concat.to_csv(r'C:\Users\Cristian.Aparicio\Documents\FAST PAYMENTS\2021\04. ABRIL\ARCHIVO RESPUESTAS FINALES\Respuesta-FP 2021-04-29.csv',index=False)

In [ ]:
# book = load_workbook('Dispersión-FP 2020-07-20.xlsx')
# writer = pd.ExcelWriter('Dispersión-FP 2020-07-20.xlsx', engine='openpyxl', options={'encoding':'utf-8'})
# writer.book = book
# match_final.to_excel(writer, sheet_name='FP-2 2020-07-20', index=False)
# writer.save()
# writer.close()